 <p style="padding:10px;background-color:BLACK;margin:0;color:white;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 15px 15px;overflow:hidden;font-weight:500">Importing a CSV file into a postgres database</p>

#### Steps
* Import the CSV file into a pandas df
* clean the table name and remove all extra symbols, spacces, capital letters
* clean the column headers and remove all extra symbol, spaces, capital letters
* write the create table SQL statement
* import the data into the db

In [1]:
import os
import numpy as np
import pandas as pd
import psycopg2

In [3]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 20EB-B907

 Directory of C:\Users\User\Desktop\SQL\Bike Store Relational Database\archive (4)

21/08/2024  10:29 AM    <DIR>          .
21/08/2024  10:29 AM    <DIR>          ..
21/08/2024  10:29 AM    <DIR>          .ipynb_checkpoints
21/08/2023  02:31 PM               120 brands.csv
21/08/2023  02:31 PM               162 categories.csv
21/08/2024  10:26 AM            14,243 csvtosql.ipynb
21/08/2023  02:31 PM           127,671 customers.csv
21/08/2023  02:31 PM           112,937 order_items.csv
21/08/2023  02:31 PM            77,286 orders.csv
21/08/2023  02:31 PM            17,390 products.csv
21/08/2023  02:31 PM               726 staffs.csv
21/08/2023  02:31 PM             8,773 stocks.csv
21/08/2023  02:31 PM               330 stores.csv
              10 File(s)        359,638 bytes
               3 Dir(s)  69,041,201,152 bytes free


In [216]:
df = pd.read_csv('stores.csv')
df.head()

,store_id,store_name,phone,email,street,city,state,zip_code
0,1,Santa Cruz Bikes,(831) 476-4321,santacruz@bikes.shop,3700 Portola Drive,Santa Cruz,CA,95060
1,2,Baldwin Bikes,(516) 379-8888,baldwin@bikes.shop,4200 Chestnut Lane,Baldwin,NY,11432
2,3,Rowlett Bikes,(972) 530-5555,rowlett@bikes.shop,8000 Fairway Avenue,Rowlett,TX,75088


In [218]:
# clean table names
# lower case letters
# remove all $
# replace -, / , \\ , whitespace with _

file = 'stores'

clean_tbl_name = file.lower().replace(' ','_').replace('?','') \
                    .replace('-','_').replace(r'/','_').replace('\\','_').replace('%','')  \
                    .replace(')','').replace(r'(','').replace('$','')

clean_tbl_name

'stores'

In [220]:
# clean table names
# lower case letters
# remove all $
# replace -, / , \\ , whitespace with _

df.columns = [x.lower().replace(' ','_').replace('?','') \
                    .replace('-','_').replace(r'/','_').replace('\\','_').replace('%','')  \
                    .replace(')','').replace(r'(','').replace('$','') for x in df.columns]
df.columns

Index(['store_id', 'store_name', 'phone', 'email', 'street', 'city', 'state',
       'zip_code'],
      dtype='object')

In [39]:
# EXAMPLE : create SQL CREATE TABLE
# (
#     customer_name          varchar,
#     start_date             varchar,
#     end_date               varchar,
#     contract_amount_m      float,
#     invoice_sent           varchar,
#     paid                   varchar
# );

In [222]:
df.dtypes

store_id       int64
store_name    object
phone         object
email         object
street        object
city          object
state         object
zip_code       int64
dtype: object

In [224]:
replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int64' : 'int',
    'datetime64' : 'timestamp',
    'timedelta64[ns]' : 'varchar'
}

replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'int',
 'datetime64': 'timestamp',
 'timedelta64[ns]': 'varchar'}

In [226]:
col_str = ', '.join('{} {}'.format(n,d) for (n,d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'store_id int, store_name varchar, phone varchar, email varchar, street varchar, city varchar, state varchar, zip_code int'

In [228]:
#open a database connection
conn_string = "host=localhost \
                port=5432 \
                dbname=bikestore \
                user=postgres \
                password=XXXXX connect_timeout=10 sslmode=prefer"

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print('Opened database successfully')

Opened database successfully


In [230]:
# drop tables with same name
cursor.execute('drop table if exists stores;')

In [232]:
# create table [just copy paste output from col_str]
cursor.execute("create table stores \
            (store_id int, store_name varchar, phone varchar, email varchar, street varchar, city varchar, state varchar, zip_code int)")

In [234]:
# insert values to table

# save df to csv
df.to_csv('stores.csv', header=df.columns, index=False, encoding='utf-8')

# open the csv file, save it as an object, and upload to db
my_file = open('stores.csv')
print('file opened in memory')

file opened in memory


In [236]:
# upload to db

SQL_STATEMENT = """
COPY stores FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""

cursor.copy_expert(sql=SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [238]:
cursor.execute('grant select on table stores to public')
conn.commit()

cursor.close()
print('table stores imported to db completed')

table stores imported to db completed
